In [1]:
from fastapi import FastAPI
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.output_parsers import StrOutputParser
import os
from dotenv import load_dotenv
from langchain_groq import ChatGroq
from  langserve import add_routes



d:\Work_Directory\LLM_Projects\documentor\.venv\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
load_dotenv()

qroq_apikey = os.getenv("GROQ_API_KEY")

model = ChatGroq(model="llama-3.1-8b-instant", groq_api_key=qroq_apikey)

In [3]:
#
from langchain_core.messages import HumanMessage
human_mesge = [HumanMessage(content="Hi, my name is albin and i am cheif ai engineer")]
model.invoke(human_mesge)

AIMessage(content="Nice to meet you, Albin. It's great to have a Chief AI Engineer like yourself here. What brings you to our platform today? Are you working on a new AI project, or do you have any questions or topics you'd like to discuss? I'm here to help and provide any information or insights I can.", additional_kwargs={}, response_metadata={'token_usage': {'completion_tokens': 67, 'prompt_tokens': 49, 'total_tokens': 116, 'completion_time': 0.104220598, 'prompt_time': 0.002289162, 'queue_time': 0.088158909, 'total_time': 0.10650976}, 'model_name': 'llama-3.1-8b-instant', 'system_fingerprint': 'fp_7b3cfae3af', 'service_tier': 'on_demand', 'finish_reason': 'stop', 'logprobs': None, 'model_provider': 'groq'}, id='lc_run--7e157699-e3ef-47f6-8a49-16e06633368f-0', usage_metadata={'input_tokens': 49, 'output_tokens': 67, 'total_tokens': 116})

In [4]:
from langchain_core.messages import AIMessage

ai_msge = [HumanMessage(content="Hi, my name is albin and i am cheif ai engineer"),
           AIMessage(content="Nice to meet you, Albin. As a Chief Operations and Maintenance (O&M) Engineer, you play a crucial role in ensuring the smooth operation and reliability of facilities, equipment, and processes. Your expertise is essential in maintaining the efficiency and productivity of the organization.\n\nWhat specific challenges or projects are you currently working on, or would you like to discuss about O&M engineering in general? "),
           HumanMessage(content="Hey, what is my name ? what i am do ?"),
           ]


model.invoke(ai_msge)

AIMessage(content='Your name is Albin, and you are the Chief AI Engineer.', additional_kwargs={}, response_metadata={'token_usage': {'completion_tokens': 15, 'prompt_tokens': 150, 'total_tokens': 165, 'completion_time': 0.024493464, 'prompt_time': 0.011025956, 'queue_time': 0.089888005, 'total_time': 0.03551942}, 'model_name': 'llama-3.1-8b-instant', 'system_fingerprint': 'fp_e32974efee', 'service_tier': 'on_demand', 'finish_reason': 'stop', 'logprobs': None, 'model_provider': 'groq'}, id='lc_run--5fba2e89-9c35-463d-b2c5-ad74f4098a3b-0', usage_metadata={'input_tokens': 150, 'output_tokens': 15, 'total_tokens': 165})

# Message History

In [5]:
from langchain_community.chat_message_histories import ChatMessageHistory
from langchain_core.chat_history import BaseChatMessageHistory
from langchain_core.runnables.history import RunnableWithMessageHistory

In [6]:
#if multoplie pepole chat with the model => how to hnadle it ==> session

store = {}

def get_session_history(session_id:str)->BaseChatMessageHistory:
    if session_id not in store:
        store[session_id] = ChatMessageHistory() # craete new one if not found if
    return store[session_id] #retrun chat associated with that session id
        
with_message_history = RunnableWithMessageHistory(model, get_session_history)

In [7]:
config1 = {"configurable" : {"session_id": "chat1"}}

In [8]:
responce = with_message_history.invoke(
    [HumanMessage(content="My name is albin, and i am learning GenAI")],
    config=config1
    )

responce.content

'Hello Albin, it\'s nice to meet you.  Learning GenAI (General Artificial Intelligence) can be a fascinating and rewarding experience. GenAI is a broad field that encompasses various aspects of AI, including machine learning, natural language processing, computer vision, and more.\n\nAs you start your journey, you might want to explore the following topics:\n\n1. **Machine Learning**: Understand the basics of machine learning, including supervised and unsupervised learning, neural networks, and deep learning.\n2. **Natural Language Processing (NLP)**: Learn about text processing, sentiment analysis, language translation, and dialogue systems.\n3. **Computer Vision**: Study image and video processing, object detection, segmentation, and recognition.\n4. **Reinforcement Learning**: Explore the concept of reinforcement learning, where an AI agent learns by interacting with an environment and receiving rewards or penalties.\n5. **Ethics and Safety**: Understand the importance of ethics and

In [9]:
#change > sessionid

config2 = {"configurable" : {"session_id": "chat2"}}
responce = with_message_history.invoke(
    [HumanMessage(content="what is my name")],
    config=config2
    )

responce.content


"I don't have information on your name. I'm a large language model, I don't have personal interactions or memories, so I don't know your name unless you tell me. If you'd like to share your name, I'd be happy to learn it."

In [10]:
#check the chat message histroy => sessionid

config3 = {"configurable" : {"session_id": "chat1"}}
responce = with_message_history.invoke(
    [HumanMessage(content="what is my name")],
    config=config3
    )

responce.content

'Your name is Albin.'

# prompt template

In [11]:
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder

In [12]:
prompt = ChatPromptTemplate.from_messages(
    [
        ("system" , "You are powerful ai model, Answer all question at your very best..."),
        MessagesPlaceholder(variable_name="messages")
        
    ]
)

chain = prompt | model
result = chain.invoke({"messages" : [HumanMessage(content="My name is Albin")]})
result.content

"Nice to meet you, Albin. I'm happy to chat with you and help with any questions or topics you'd like to discuss. How's your day going so far?"

In [13]:
# chat history
with_msge_history = RunnableWithMessageHistory(chain, get_session_history)

config = {"configurable" : {"session_id" : "chat3"}}
res  = with_message_history.invoke(
        [HumanMessage(content="My name is Albin")],
        config=config
    )
res.content

"Nice to meet you, Albin. How can I assist you today? Do you have any questions or topics you'd like to discuss?"

In [14]:
## add more complexity
prompt = ChatPromptTemplate.from_messages(
    [
        ("system" , "You are powerful ai model, Answer all question at your very best in {language}"),
        MessagesPlaceholder(variable_name="messages"),
        
    ]
)

chain = prompt | model
result = chain.invoke(
    {
        "messages" : [HumanMessage(content="My name is Albin")], 
        "language" : "German"
    }
)
result.content

'Hallo Albin, ich bin froh, dich kennenzulernen. Wie kann ich dir heute helfen? Hast du Fragen oder möchtest du einfach nur reden?'

In [15]:
# chat history
with_msge_history = RunnableWithMessageHistory(chain, get_session_history, 
                                    input_messages_key="input")

config = {"configurable": {"session_id": "chat_mueller_1"}}
res2  = with_message_history.invoke(
    {
        "input" : [HumanMessage(content="My name is Albin")], 
        "language" : "German"
    },
        config=config 
    )
res2.content

'Nice to meet you, Albin. Is there something I can help you with or would you like to chat?'

# Manage Chat Histroy

In [16]:
from langchain_core.messages import SystemMessage, trim_messages

# ' trim_messages' = reduce how many message we are sending to the model

In [17]:
trimmer = trim_messages(
    max_tokens=400,          #should be exact ´same params name
    strategy="last",        #should be exact ´same params name
    token_counter=model,
    include_system=True,
    allow_partial=False,
    start_on="human"
)

messages = [
    SystemMessage(content="You are the good assistant"),
    HumanMessage(content="Hi! I am Müller"),
    AIMessage(content="Hai!!"),
    HumanMessage(content="i Like vanilla ice cream"),
    AIMessage(content="oh that nice"),
    HumanMessage(content="what is 9+2"),
    AIMessage(content="that 11 Müller"),
    HumanMessage(content="thank you"),
    AIMessage(content="NO probelm"),
]

trimmer.invoke(messages)


[SystemMessage(content='You are the good assistant', additional_kwargs={}, response_metadata={}),
 HumanMessage(content='Hi! I am Müller', additional_kwargs={}, response_metadata={}),
 AIMessage(content='Hai!!', additional_kwargs={}, response_metadata={}),
 HumanMessage(content='i Like vanilla ice cream', additional_kwargs={}, response_metadata={}),
 AIMessage(content='oh that nice', additional_kwargs={}, response_metadata={}),
 HumanMessage(content='what is 9+2', additional_kwargs={}, response_metadata={}),
 AIMessage(content='that 11 Müller', additional_kwargs={}, response_metadata={}),
 HumanMessage(content='thank you', additional_kwargs={}, response_metadata={}),
 AIMessage(content='NO probelm', additional_kwargs={}, response_metadata={})]

In [21]:
# how to pass trimmer in a chain
from operator import itemgetter
from langchain_core.runnables import RunnablePassthrough


trimmed = RunnablePassthrough.assign(messages=itemgetter("messages") | trimmer)
chain = trimmed | prompt | model

response  = chain.invoke({
    "messages" : messages + [HumanMessage(content= "what is 9+2 ? ")],
    "language" : "English"
}
)

response.content

"I already told you Müller, it's 11!"

In [22]:
# wrpa message hiostory
# chat history
with_msge_history = RunnableWithMessageHistory(chain, get_session_history, input_messages_key="messages")

config = {"configurable" : {"session_id" : "chat5"}}
res2  = with_msge_history.invoke(
    {
        "messages" : [HumanMessage(content="what math question i asked")], 
        "language" : "englsih"
    },
        config=config 
    )
res2.content

'You asked this question: \n\n"what math question i asked"\n\nHowever, this is not a math question, it\'s a general question asking if you had asked a math question, which you hadn\'t.'

## Vector Retriver

In [23]:
from langchain_core.documents import Document

document = [
    Document(
        page_content="Cats are independent pets that often enjoy their own space.",
        metadata={"source": "mammal-pets-doc"},
    ),
    Document(
        page_content="Goldfish are popular pets for beginners, requiring relatively simple care.",
        metadata={"source": "fish-pets-doc"},
    ),
    Document(
        page_content="Parrots are intelligent birds capable of mimicking human speech.",
        metadata={"source": "bird-pets-doc"},
    ),
    Document(
        page_content="Rabbits are social animals that need plenty of space to hop around.",
        metadata={"source": "mammal-pets-doc"},
    ),
]


In [25]:
os.environ["HF_TOKEN"] = os.getenv("HF_TOKEN")

In [28]:
qroq_apikey = os.getenv("GROQ_API_KEY")

llm_model = ChatGroq(model="llama-3.1-8b-instant", groq_api_key=qroq_apikey)
llm_model

ChatGroq(client=<groq.resources.chat.completions.Completions object at 0x000001ED1C390DF0>, async_client=<groq.resources.chat.completions.AsyncCompletions object at 0x000001ED1C391450>, model_name='llama-3.1-8b-instant', model_kwargs={}, groq_api_key=SecretStr('**********'))

In [29]:
from langchain_huggingface import HuggingFaceEmbeddings
embeddings = HuggingFaceEmbeddings(model="all-MiniLM-L6-v2")

In [30]:
#convert word into embedding
from langchain_chroma import Chroma

vectorstore = Chroma.from_documents(document, embedding=embeddings)

In [31]:
# similarity search
vectorstore.similarity_search("cat")

[Document(id='e8217fba-4e9b-42ca-834e-bbb34046da02', metadata={'source': 'mammal-pets-doc'}, page_content='Cats are independent pets that often enjoy their own space.'),
 Document(id='cc7607bd-0922-44ad-ad12-68f961b460ea', metadata={'source': 'mammal-pets-doc'}, page_content='Rabbits are social animals that need plenty of space to hop around.'),
 Document(id='2c5c9064-8685-4648-8711-87b5fda13b4f', metadata={'source': 'bird-pets-doc'}, page_content='Parrots are intelligent birds capable of mimicking human speech.'),
 Document(id='3d3548c2-26e9-40ad-af94-2244a57ba20c', metadata={'source': 'fish-pets-doc'}, page_content='Goldfish are popular pets for beginners, requiring relatively simple care.')]

In [32]:
#similarity seach with score
vectorstore.similarity_search_with_score("cat")

[(Document(id='e8217fba-4e9b-42ca-834e-bbb34046da02', metadata={'source': 'mammal-pets-doc'}, page_content='Cats are independent pets that often enjoy their own space.'),
  0.9351058006286621),
 (Document(id='cc7607bd-0922-44ad-ad12-68f961b460ea', metadata={'source': 'mammal-pets-doc'}, page_content='Rabbits are social animals that need plenty of space to hop around.'),
  1.5956906080245972),
 (Document(id='2c5c9064-8685-4648-8711-87b5fda13b4f', metadata={'source': 'bird-pets-doc'}, page_content='Parrots are intelligent birds capable of mimicking human speech.'),
  1.6657925844192505),
 (Document(id='3d3548c2-26e9-40ad-af94-2244a57ba20c', metadata={'source': 'fish-pets-doc'}, page_content='Goldfish are popular pets for beginners, requiring relatively simple care.'),
  1.8011280298233032)]

# Retriver

In [34]:
from typing import List
from langchain_core.documents import Document
from langchain_core.runnables import RunnableLambda

retriver = RunnableLambda(vectorstore.similarity_search).bind(k=1)
retriver.batch(["cat", "Goldfish"])


[[Document(id='e8217fba-4e9b-42ca-834e-bbb34046da02', metadata={'source': 'mammal-pets-doc'}, page_content='Cats are independent pets that often enjoy their own space.')],
 [Document(id='3d3548c2-26e9-40ad-af94-2244a57ba20c', metadata={'source': 'fish-pets-doc'}, page_content='Goldfish are popular pets for beginners, requiring relatively simple care.')]]

In [36]:
# method 2 - best way
ret = vectorstore.as_retriever(
    search_type = "similarity",
    search_kwarg = {"k": 1}
)

ret.batch(["cat", "goldfish"])


[[Document(id='e8217fba-4e9b-42ca-834e-bbb34046da02', metadata={'source': 'mammal-pets-doc'}, page_content='Cats are independent pets that often enjoy their own space.'),
  Document(id='cc7607bd-0922-44ad-ad12-68f961b460ea', metadata={'source': 'mammal-pets-doc'}, page_content='Rabbits are social animals that need plenty of space to hop around.'),
  Document(id='2c5c9064-8685-4648-8711-87b5fda13b4f', metadata={'source': 'bird-pets-doc'}, page_content='Parrots are intelligent birds capable of mimicking human speech.'),
  Document(id='3d3548c2-26e9-40ad-af94-2244a57ba20c', metadata={'source': 'fish-pets-doc'}, page_content='Goldfish are popular pets for beginners, requiring relatively simple care.')],
 [Document(id='3d3548c2-26e9-40ad-af94-2244a57ba20c', metadata={'source': 'fish-pets-doc'}, page_content='Goldfish are popular pets for beginners, requiring relatively simple care.'),
  Document(id='2c5c9064-8685-4648-8711-87b5fda13b4f', metadata={'source': 'bird-pets-doc'}, page_content='P